In [27]:
class Monad m => MonadReader r m where
  ask   :: m r
  local :: (r -> r) -> m a -> m a

In [49]:
newtype Reader r a = Reader { runReader :: r -> a }

-- class Reader r a

instance Functor (Reader r) where
    fmap :: (a -> b) -> Reader r a -> Reader r b
    fmap f (Reader ra) = Reader (f . ra)

instance Applicative (Reader r) where
    pure :: a -> Reader r a
    pure x = Reader (\_ -> x)
    (<*>) :: Reader r (a -> b) -> Reader r a -> Reader r b
    Reader f <*> Reader ra = Reader (\r -> f r (ra r))
    -- Reader r -> a -> b <*> Reader r -> a

instance Monad (Reader r) where
    return :: a -> Reader r a
    return = pure

    (>>=) :: Reader r a -> (a -> Reader r b) -> Reader r b
    Reader ra >>= f = Reader (\r -> runReader (f $ ra r) r)



Line 11: Use const
Found:
\ _ -> x
Why not:
const x

In [64]:

instance MonadReader r (Reader r) where
    ask :: Reader r r
    ask = Reader (\r -> r)
    
    local :: (r -> r) -> Reader r a -> Reader r a
    local f m = Reader (\r -> runReader m $ f r)

Line 4: Use id
Found:
\ r -> r
Why not:
idLine 7: Avoid lambda
Found:
\ r -> runReader m $ f r
Why not:
runReader m . f

In [86]:
type Env a b = a -> Maybe b

data Exp = Num Int
         | Exp :+: Exp
         | Exp :/: Exp
         | Var String
         | Let String Int Exp
         
type ExpEnv = Env String Int

insert :: Eq a => a -> b -> Env a b -> Env a b
insert k v env k' = if k == k' then Just v else env k'

emptyEnv :: ExpEnv
emptyEnv = const Nothing

In [88]:
evalReader :: MonadReader ExpEnv m => Exp -> m (Maybe Int)
evalReader (Num x) = return $ Just x
evalReader (e1 :+: e2) = evalReader e1 >>= \ee1 -> evalReader e2 >>= \ee2 -> return ((+) <$> ee1 <*> ee2)
evalReader (e1 :/: e2) = evalReader e2 >>= \ee2 -> (case ee2 of
                                                        Just 0 -> return Nothing 
                                                        other -> evalReader e1 >>= \ee1 -> return (div <$> ee1 <*> ee2))
evalReader (Var s) = ask >>= \env -> return $ env s
evalReader (Let s i e) = evalReader e >>= \ee -> ask >>= \env -> local (insert s i) $ evalReader e


: 